## Code for SDCCA with examples

In [7]:
import pandas as pd
import numpy as np
import math
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

In [45]:
def _center_scale_xy(x, y, scale=True):
    
    #Normalizes the data using StandardScaler
    
    if scale == True:
        return StandardScaler().fit_transform(x),StandardScaler().fit_transform(y)
    return x,y

class SDCCA :

    """
    Similarity Distance based CCA (SDCCA)
    
    Parameters
    ----------
    scale (boolean) (default true) : whether to scale the data or not    
    k (integer) (default 2) : number of neighbours to consider in K-Means algorithm.

    Attributes
    ----------
    x : first data set of dimension n x p with n samples and p features.
    y : second data set of dimension n x q with n samples and q features.    
    wx , wy : final projection vectors of two views of dimensions p x p and q x q respectively.
    n : number of samples in datasets
    p : number of features in x dataset
    q : number of features in y dataset
    
    """
    
    def __init__(self,scale = True,k=2):
        self.scale = scale
        self.k = k
        
    def __make_dimensionlity_same(self,x,y,p,q) :
        
        #Uses PCA to make the number of features same in both the dataset
        
        pca = decomposition.PCA(n_components=min(p,q))
        if p < q : 
            pc = pca.fit_transform(y)
            return x,pc,p,p
        else : 
            pc = pca.fit_transform(x)
            return pc,y,q,q
        
    def __bhattacharya_similarity_coeff(self,x,y) : 
        
        #Calculates the Bhattacharya similarity Coefficient

        n = x.shape[0]
        p = x.shape[1]
        q = y.shape[1]

        if p != q :
            x,y,p,q = self.__make_dimensionlity_same(x,y,p,q)

        sxy = np.zeros((n,n))
        for i in range(n) :
            for j in range(n) : 
                sxy[i][j] = abs(np.dot(x[i],y[j]))**0.5
        return sxy
    
    def __nearest_neighbor(self,x,y) :
        
        # Uses K-Means Algorithm to find the K neighbours and correspnding distances for all the samples.
        
        neigh = NearestNeighbors(n_neighbors=self.k)
        neigh.fit(x)
        x_dist, x_neigh = neigh.kneighbors(x,return_distance = True)
        
        neigh.fit(y)
        y_dist, y_neigh = neigh.kneighbors(y,return_distance = True)
        
        return x_dist,x_neigh,y_dist,y_neigh
            
    def fit(self,x,y) :

        """
        Fit the model from the data in x and the labels in y and finds the projection vectors
        
        Parameters
        ----------
        x : numpy array-like, shape (n x p)
            where n is the number of samples, and p is the number of features.
        y : numpy array-like, shape (n x q)
            where n is the number of samples, and q is the number of features.
            
        Variables
        ----------
        x_neigh , y_neigh : indices of k neighbours of each of the n samples of x and y respectively computed using K-Means algorithm.
        x_dist, y_dist : dist from each of the k neighbours of all n samples of x and y respectively.

        sx, sy : local manifold information between samples.
        sxy : it is dissimilarity between samples using 1-s~xy.
        s~xy : it is similarity between views x and y calculated using Bhattacharya similarity coefficient.
        
        Returns
        -------
        wx , wy : Projection vectors of dimensions p x p and q x q respectively.
      """
        # n is the number of samples and p and q are features of x and y respectively.
        self.n = x.shape[0]
        self.p = x.shape[1]
        self.q = y.shape[1]
        
        self.x = x
        self.y = y
                    
        # normalize the data
        self.x, self.y = _center_scale_xy(self.x, self.y, self.scale)
        
        x_dist,x_neigh,y_dist,y_neigh = self.__nearest_neighbor(self.x,self.y)
        
        tx = 0
        ty = 0
        sx = np.zeros((self.n,self.n))
        sy = np.zeros((self.n,self.n))
        
        for i in range(self.n) : 
            for j in range(self.n) : 
                tx += (2*(np.linalg.norm(self.x[i]-self.x[j],2)**2))/(self.n*(self.n-1))
        
        for i in range(self.n) : 
            for j in range(self.n) : 
                ty += (2*(np.linalg.norm(self.y[i]-self.y[j],2)**2))/(self.n*(self.n-1))
                
        for i in range(self.n) : 
            for j in range(self.k) : 
                sx[i][x_neigh[i][j]] = np.exp(-(x_dist[i][j]**2)/tx)

        for i in range(self.n) : 
            for j in range(self.k) : 
                sy[i][y_neigh[i][j]] = np.exp(-(y_dist[i][j]**2)/ty)
                                    
        sxy = 1-self.__bhattacharya_similarity_coeff(self.x,self.y)
        
        s = np.identity(self.n)+sx+sy+sxy

        cxy = np.dot(np.dot(self.x.transpose(),s),self.y) 
        cxx = np.dot(self.x.transpose(),self.x)
        cyy = np.dot(self.y.transpose(),self.y,)
        
        temp = np.dot(cxx, cxy)
        h = np.dot(temp,cyy)

        # Use SVD(singular value decomposition) to find the U and V.T matrix
        u,d,vh = np.linalg.svd(h)

        #Calculate projection vectors
        self.wx = np.dot(cxx, u)
        self.wy = np.dot(cyy, vh.transpose())
        
        return self.wx,self.wy
    
    def fit_transform(self, x,y) :

        """
        Applies the projection vectors on the dataset
        ----------
        x : numpy array-like, shape (n x p)
            where n is the number of samples, and p is the number of features.
        y : numpy array-like, shape (n x q)
            where n is the number of samples, and q is the number of features.
        
        Returns
        -------
        x_new , y_new : Projected views
      """
        wx,wy = self.fit(x,y)
        x_new = np.dot(x,wx.T)
        y_new = np.dot(y,wy.T)
        return x_new,y_new

## Example 1

In [49]:
x = np.array([[0., 0., 1.], [1.,0.,0.], [2.,2.,2.], [3.,5.,4.]])
y = np.array([[0.1, 0.2], [0.9, 1.1], [6.2, 5.9], [11.9, 12.3]])
sdcca = SDCCA(scale = True) # k is by default 2
wx, wy = sdcca.fit(x,y)
np.set_printoptions(precision = 7, suppress = True)
print("wx")
print(wx)
print("wy")
print(wy)

wx
[[-6.3707882  0.5281025 -0.0026756]
 [-6.6984369 -0.0751526 -0.0186668]
 [-6.476623  -0.4333642  0.0218077]]
wy
[[-5.6531055  0.0037124]
 [-5.6531055 -0.0037851]]


## Example 2

In [43]:
x = np.array([[1., 2.1, 1.], [1., 2.1, 0.5], [2.,2.,2.], [3.,5.,4.]])
y = np.array([[0.1, 4.2], [0.9, 2.1], [7.2, 5.9], [12.9, 1.3]])
sdcca = SDCCA(k=4)
x_new, y_new = sdcca.fit_transform(x,y)
print("X New")
print(x_new)
print("Y New")
print(y_new)

X New
[[ -5.8380473  -7.4261078  -6.2538804]
 [ -5.823948   -7.4249809  -6.2690812]
 [-12.4623148 -13.7376682 -12.9121544]
 [-18.0069253 -21.6219808 -18.9701994]]
Y New
[[  3.896946   12.9154699]
 [ -1.4534688   8.8647358]
 [-22.7799413  38.134078 ]
 [-50.2993919  40.4397741]]
